# <span style="color:#4e5ec5">Import Packages</span>


In [121]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
from datetime import datetime
import calendar
from math import sin, cos, sqrt, atan2, radians,asin
import folium
from folium import FeatureGroup, LayerControl, Map, Marker
from folium.plugins import HeatMap
from folium.plugins import TimestampedGeoJson
from folium.plugins import MarkerCluster
from geopy.distance import great_circle
import matplotlib.dates as mdates
import matplotlib as mpl
from datetime import timedelta
import datetime as dt
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', -1)
plt.style.use('fivethirtyeight')
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import pickle
from geopy.distance import geodesic
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgbm
import xgboost as xgb 

%matplotlib inline




# <span style="color:#4e5ec5">Load Train and Test Datasets</span>


In [122]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
train.head()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
0,189123628,10.5,834,56,0.0000,64,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,correct
1,189125358,10.5,791,47,0.0000,134,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,correct
2,189125719,10.5,1087,80,0.0000,61,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,correct
3,189127273,10.5,598,271,15.6638,68,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,correct
4,189129552,10.5,3407,182,0.0000,112,11/1/2019 5:38,11/1/2019 6:35,7.13402,79.8969,6.91865,79.8649,1065.02,correct


In [123]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16967 entries, 0 to 16966
Data columns (total 14 columns):
tripid                       16967 non-null int64
additional_fare              16967 non-null float64
duration                     16967 non-null int64
meter_waiting                16967 non-null int64
meter_waiting_fare           16967 non-null float64
meter_waiting_till_pickup    16967 non-null int64
pickup_time                  16967 non-null object
drop_time                    16967 non-null object
pick_lat                     16967 non-null float64
pick_lon                     16967 non-null float64
drop_lat                     16967 non-null float64
drop_lon                     16967 non-null float64
fare                         16967 non-null float64
label                        16967 non-null object
dtypes: float64(7), int64(4), object(3)
memory usage: 1.8+ MB


# <span style="color:#4e5ec5">Pre process pickup and drop off times</span>

### <span style="color:#4ec57a"> Remove outliers</span>

In [124]:
#train = train.loc[train['duration'] > 0]

In [125]:
#train = train.loc[train['fare'] > 0]

In [126]:
#train = train.loc[train['fare'] < 11000]

In [127]:
#train = train.loc[train['meter_waiting_till_pickup'] < 10000]
#train = train.loc[train['meter_waiting_fare'] < 10000]



### <span style="color:#4ec57a"> Convert pickup and drop off time columns to date type. Split the Pickup time and Drop time in to year-month-day-day of week-hour</span>

In [128]:
train['pickup_time']=pd.to_datetime(train['pickup_time'])
train['drop_time']=pd.to_datetime(train['drop_time'])
train.head()

test['pickup_time']=pd.to_datetime(test['pickup_time'])
test['drop_time']=pd.to_datetime(test['drop_time'])

In [129]:
# beak the pickup date time
train['pickup_date']= train['pickup_time'].dt.date
train['pickup_day']=train['pickup_time'].apply(lambda x:x.day)
train['pickup_hour']=train['pickup_time'].apply(lambda x:x.hour)
train['pickup_day_of_week']=train['pickup_time'].apply(lambda x:x.dayofweek)
train['pickup_month']=train['pickup_time'].apply(lambda x:x.month)
train['pickup_year']=train['pickup_time'].apply(lambda x:x.year)

test['pickup_date']= test['pickup_time'].dt.date
test['pickup_day']=test['pickup_time'].apply(lambda x:x.day)
test['pickup_hour']=test['pickup_time'].apply(lambda x:x.hour)
test['pickup_day_of_week']=test['pickup_time'].apply(lambda x:x.dayofweek)
test['pickup_month']=test['pickup_time'].apply(lambda x:x.month)
test['pickup_year']=test['pickup_time'].apply(lambda x:x.year)

# beak the drop off date time
train['drop_date']= train['drop_time'].dt.date
train['drop_day']=train['drop_time'].apply(lambda x:x.day)
train['drop_hour']=train['drop_time'].apply(lambda x:x.hour)
train['drop_day_of_week']=train['drop_time'].apply(lambda x:x.dayofweek)
train['drop_month']=train['drop_time'].apply(lambda x:x.month)
train['drop_year']=train['drop_time'].apply(lambda x:x.year)

test['drop_date']= test['drop_time'].dt.date
test['drop_day']=test['drop_time'].apply(lambda x:x.day)
test['drop_hour']=test['drop_time'].apply(lambda x:x.hour)
test['drop_day_of_week']=test['drop_time'].apply(lambda x:x.dayofweek)
test['drop_month']=test['drop_time'].apply(lambda x:x.month)
test['drop_year']=test['drop_time'].apply(lambda x:x.year)

train.head()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,...,pickup_hour,pickup_day_of_week,pickup_month,pickup_year,drop_date,drop_day,drop_hour,drop_day_of_week,drop_month,drop_year
0,189123628,10.5,834,56,0.0000,64,2019-11-01 00:20:00,2019-11-01 00:34:00,6.86252,79.8993,...,0,4,11,2019,2019-11-01,1,0,4,11,2019
1,189125358,10.5,791,47,0.0000,134,2019-11-01 00:56:00,2019-11-01 01:09:00,6.88589,79.8984,...,0,4,11,2019,2019-11-01,1,1,4,11,2019
2,189125719,10.5,1087,80,0.0000,61,2019-11-01 01:08:00,2019-11-01 01:26:00,6.90839,79.8651,...,1,4,11,2019,2019-11-01,1,1,4,11,2019
3,189127273,10.5,598,271,15.6638,68,2019-11-01 02:27:00,2019-11-01 02:37:00,6.92570,79.8895,...,2,4,11,2019,2019-11-01,1,2,4,11,2019
4,189129552,10.5,3407,182,0.0000,112,2019-11-01 05:38:00,2019-11-01 06:35:00,7.13402,79.8969,...,5,4,11,2019,2019-11-01,1,6,4,11,2019


### <span style="color:#4ec57a">New column to hold label as boolean value</span>

In [130]:
train['label_bool']=train['label'].apply(lambda x:x=='correct')

train.head()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,...,pickup_day_of_week,pickup_month,pickup_year,drop_date,drop_day,drop_hour,drop_day_of_week,drop_month,drop_year,label_bool
0,189123628,10.5,834,56,0.0000,64,2019-11-01 00:20:00,2019-11-01 00:34:00,6.86252,79.8993,...,4,11,2019,2019-11-01,1,0,4,11,2019,True
1,189125358,10.5,791,47,0.0000,134,2019-11-01 00:56:00,2019-11-01 01:09:00,6.88589,79.8984,...,4,11,2019,2019-11-01,1,1,4,11,2019,True
2,189125719,10.5,1087,80,0.0000,61,2019-11-01 01:08:00,2019-11-01 01:26:00,6.90839,79.8651,...,4,11,2019,2019-11-01,1,1,4,11,2019,True
3,189127273,10.5,598,271,15.6638,68,2019-11-01 02:27:00,2019-11-01 02:37:00,6.92570,79.8895,...,4,11,2019,2019-11-01,1,2,4,11,2019,True
4,189129552,10.5,3407,182,0.0000,112,2019-11-01 05:38:00,2019-11-01 06:35:00,7.13402,79.8969,...,4,11,2019,2019-11-01,1,6,4,11,2019,True


### check for missing values

In [131]:
train[pd.isnull(train)].sum()
test[pd.isnull(test)].sum()

tripid                       0.0
additional_fare              0.0
duration                     0.0
meter_waiting                0.0
meter_waiting_fare           0.0
meter_waiting_till_pickup    0.0
pickup_time                  0.0
drop_time                    0.0
pick_lat                     0.0
pick_lon                     0.0
drop_lat                     0.0
drop_lon                     0.0
fare                         0.0
pickup_date                  0.0
pickup_day                   0.0
pickup_hour                  0.0
pickup_day_of_week           0.0
pickup_month                 0.0
pickup_year                  0.0
drop_date                    0.0
drop_day                     0.0
drop_hour                    0.0
drop_day_of_week             0.0
drop_month                   0.0
drop_year                    0.0
dtype: float64

In [132]:
train.describe()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,...,pickup_day,pickup_hour,pickup_day_of_week,pickup_month,pickup_year,drop_day,drop_hour,drop_day_of_week,drop_month,drop_year
count,1.696700e+04,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,...,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000
mean,2.014702e+08,13.721627,1702.918901,628.909118,32.046838,112.490364,6.965130,79.997982,6.965852,79.999153,...,15.859433,13.803029,2.919785,7.949844,2019.339836,15.860376,14.082277,2.921730,7.948665,2019.339954
std,7.453342e+06,98.995408,8121.788501,8064.894289,465.577907,442.142959,0.210811,0.242792,0.211475,0.242978,...,8.843951,4.819491,1.940130,5.002983,0.473667,8.844053,4.881715,1.940802,5.003443,0.473707
min,1.891236e+08,0.000000,0.000000,0.000000,0.000000,0.000000,5.937600,79.819100,5.941860,79.796700,...,1.000000,0.000000,0.000000,1.000000,2019.000000,1.000000,0.000000,0.000000,1.000000,2019.000000
25%,1.947352e+08,10.500000,548.000000,33.000000,0.000000,26.000000,6.864830,79.870450,6.865580,79.870100,...,8.000000,10.000000,1.000000,1.000000,2019.000000,8.000000,11.000000,1.000000,1.000000,2019.000000
50%,2.013933e+08,10.500000,1009.000000,123.000000,0.115600,73.000000,6.913970,79.898600,6.915930,79.899000,...,16.000000,14.000000,3.000000,11.000000,2019.000000,16.000000,14.000000,3.000000,11.000000,2019.000000
75%,2.083317e+08,10.500000,1802.500000,351.000000,14.334400,144.000000,7.042645,79.956450,7.046540,79.960000,...,23.000000,18.000000,5.000000,12.000000,2020.000000,23.000000,18.000000,5.000000,12.000000,2020.000000
max,2.138173e+08,12795.030000,453872.000000,453650.000000,26198.601400,51288.000000,8.051010,81.009600,8.064930,81.009700,...,31.000000,23.000000,6.000000,12.000000,2020.000000,31.000000,23.000000,6.000000,12.000000,2020.000000


### <span style="color:#4ec57a">Function created to calculate distance based on longitudes and latitudes of pickup and drop off locations</span>

In [133]:
def haversine_distance(lat1, long1, lat2, long2):
    data = [train, test]
    for i in data:
        R = 6371  #radius of earth in kilometers
        #R = 3959 #radius of earth in miles
        phi1 = np.radians(i[lat1])
        phi2 = np.radians(i[lat2])
    
        delta_phi = np.radians(i[lat2]-i[lat1])
        delta_lambda = np.radians(i[long2]-i[long1])
    
        #a = sin²((φB - φA)/2) + cos φA . cos φB . sin²((λB - λA)/2)
        a = np.sin(delta_phi / 2.0) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2.0) ** 2
    
        #c = 2 * atan2( √a, √(1−a) )
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    
        #d = R*c
        d = (R * c) #in kilometers
        i['H_Distance'] = d
    return d

In [134]:
haversine_distance('pick_lat', 'pick_lon', 'drop_lat', 'drop_lon')
train['H_Distance'].head(10)

0    5.092770 
1    3.168058 
2    6.305395 
3    0.861946 
4    24.207039
5    4.777624 
6    5.322544 
7    1.035302 
8    2.930715 
9    14.381001
Name: H_Distance, dtype: float64

In [135]:
train.describe()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,...,pickup_hour,pickup_day_of_week,pickup_month,pickup_year,drop_day,drop_hour,drop_day_of_week,drop_month,drop_year,H_Distance
count,1.696700e+04,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,...,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000
mean,2.014702e+08,13.721627,1702.918901,628.909118,32.046838,112.490364,6.965130,79.997982,6.965852,79.999153,...,13.803029,2.919785,7.949844,2019.339836,15.860376,14.082277,2.921730,7.948665,2019.339954,4.524499
std,7.453342e+06,98.995408,8121.788501,8064.894289,465.577907,442.142959,0.210811,0.242792,0.211475,0.242978,...,4.819491,1.940130,5.002983,0.473667,8.844053,4.881715,1.940802,5.003443,0.473707,4.602826
min,1.891236e+08,0.000000,0.000000,0.000000,0.000000,0.000000,5.937600,79.819100,5.941860,79.796700,...,0.000000,0.000000,1.000000,2019.000000,1.000000,0.000000,0.000000,1.000000,2019.000000,0.000000
25%,1.947352e+08,10.500000,548.000000,33.000000,0.000000,26.000000,6.864830,79.870450,6.865580,79.870100,...,10.000000,1.000000,1.000000,2019.000000,8.000000,11.000000,1.000000,1.000000,2019.000000,1.486029
50%,2.013933e+08,10.500000,1009.000000,123.000000,0.115600,73.000000,6.913970,79.898600,6.915930,79.899000,...,14.000000,3.000000,11.000000,2019.000000,16.000000,14.000000,3.000000,11.000000,2019.000000,3.216324
75%,2.083317e+08,10.500000,1802.500000,351.000000,14.334400,144.000000,7.042645,79.956450,7.046540,79.960000,...,18.000000,5.000000,12.000000,2020.000000,23.000000,18.000000,5.000000,12.000000,2020.000000,6.150903
max,2.138173e+08,12795.030000,453872.000000,453650.000000,26198.601400,51288.000000,8.051010,81.009600,8.064930,81.009700,...,23.000000,6.000000,12.000000,2020.000000,31.000000,23.000000,6.000000,12.000000,2020.000000,79.220774


In [136]:
def late_night (row):
    if (row['pickup_hour'] <= 6) or (row['pickup_hour'] >= 22):
        return 1
    else:
        return 0


def rush_hours (row, column):
    if ((row[column] <= 20) and (row[column] >= 18)) and (row['pickup_day_of_week'] < 5):
        return 1
    else:
        return 0

train['rush_hours']=train.apply (lambda x: rush_hours(x, 'pickup_hour'), axis=1)
test['rush_hours']= test.apply (lambda x: rush_hours(x, 'pickup_hour'), axis=1)

train['late_night']=train.apply (lambda x: late_night(x), axis=1)
test['late_night']= test.apply (lambda x: late_night(x), axis=1)

In [137]:
train.head(50)

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,...,pickup_month,pickup_year,drop_date,drop_day,drop_hour,drop_day_of_week,drop_month,drop_year,label_bool,H_Distance
0,189123628,10.5,834,56,0.000000,64,2019-11-01 00:20:00,2019-11-01 00:34:00,6.86252,79.8993,...,11,2019,2019-11-01,1,0,4,11,2019,True,5.092770
1,189125358,10.5,791,47,0.000000,134,2019-11-01 00:56:00,2019-11-01 01:09:00,6.88589,79.8984,...,11,2019,2019-11-01,1,1,4,11,2019,True,3.168058
2,189125719,10.5,1087,80,0.000000,61,2019-11-01 01:08:00,2019-11-01 01:26:00,6.90839,79.8651,...,11,2019,2019-11-01,1,1,4,11,2019,True,6.305395
3,189127273,10.5,598,271,15.663800,68,2019-11-01 02:27:00,2019-11-01 02:37:00,6.92570,79.8895,...,11,2019,2019-11-01,1,2,4,11,2019,True,0.861946
4,189129552,10.5,3407,182,0.000000,112,2019-11-01 05:38:00,2019-11-01 06:35:00,7.13402,79.8969,...,11,2019,2019-11-01,1,6,4,11,2019,True,24.207039
5,189132829,10.5,1246,487,0.000000,133,2019-11-01 06:29:00,2019-11-01 06:49:00,6.84371,79.9051,...,11,2019,2019-11-01,1,6,4,11,2019,True,4.777624
6,189135103,10.5,1333,295,17.198500,212,2019-11-01 06:50:00,2019-11-01 07:12:00,6.90760,79.9524,...,11,2019,2019-11-01,1,7,4,11,2019,True,5.322544
7,189139296,10.5,360,80,4.664000,3,2019-11-01 07:00:00,2019-11-01 07:06:00,7.26706,80.6064,...,11,2019,2019-11-01,1,7,4,11,2019,True,1.035302
8,189138671,10.5,1539,588,33.986400,43,2019-11-01 07:02:00,2019-11-01 07:28:00,6.85137,79.9537,...,11,2019,2019-11-01,1,7,4,11,2019,True,2.930715
9,189135814,10.5,26,19,0.000000,73,2019-11-01 07:27:00,2019-11-01 07:27:00,7.00355,79.8819,...,11,2019,2019-11-01,1,7,4,11,2019,False,14.381001


In [138]:
#train['long_dist'] = train['pick_lat'] - train['drop_lat']
#train['lat_dist'] = train['pick_lon'] - train['drop_lon']

#test['long_dist'] = test['pick_lat'] - test['drop_lat']
#test['lat_dist'] = test['pick_lon'] - test['drop_lon']

In [139]:
train.columns

Index(['tripid', 'additional_fare', 'duration', 'meter_waiting',
       'meter_waiting_fare', 'meter_waiting_till_pickup', 'pickup_time',
       'drop_time', 'pick_lat', 'pick_lon', 'drop_lat', 'drop_lon', 'fare',
       'label', 'pickup_date', 'pickup_day', 'pickup_hour',
       'pickup_day_of_week', 'pickup_month', 'pickup_year', 'drop_date',
       'drop_day', 'drop_hour', 'drop_day_of_week', 'drop_month', 'drop_year',
       'label_bool', 'H_Distance'],
      dtype='object')

In [140]:
test.columns

Index(['tripid', 'additional_fare', 'duration', 'meter_waiting',
       'meter_waiting_fare', 'meter_waiting_till_pickup', 'pickup_time',
       'drop_time', 'pick_lat', 'pick_lon', 'drop_lat', 'drop_lon', 'fare',
       'pickup_date', 'pickup_day', 'pickup_hour', 'pickup_day_of_week',
       'pickup_month', 'pickup_year', 'drop_date', 'drop_day', 'drop_hour',
       'drop_day_of_week', 'drop_month', 'drop_year', 'H_Distance'],
      dtype='object')

In [141]:
no_0 = train.loc[train['duration'] == 0] 

In [142]:
no_0.head()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,...,pickup_month,pickup_year,drop_date,drop_day,drop_hour,drop_day_of_week,drop_month,drop_year,label_bool,H_Distance
3518,193653017,0.0,0,0,0.0,540,2019-11-21 15:57:00,2019-11-21 16:23:00,6.90471,79.8520,...,11,2019,2019-11-21,21,16,3,11,2019,False,0.019084
16545,213112837,0.0,0,0,0.0,973,2020-01-29 20:00:00,2020-01-29 20:23:00,6.85275,79.9391,...,1,2020,2020-01-29,29,20,2,1,2020,False,0.842827


# <span style="color:#c54e5e">Method 1: Classification Using Random Forests</span>

##### <span style="color:#4ec57a">In this method I'm trying to predict the correctnes value which will be between 0 and 1. Prediction will be evaluated based on the value >= 0.5 or not</span>

### <span style="color:#4ec57a">Copy of cleaned data set

In [143]:
C_train = train.drop(['tripid','pickup_time', 'drop_time', 'pickup_date', 'drop_date', 'label', 'pick_lat', 'pick_lon', 'drop_lat', 'drop_lon'], axis = 1)
C_test = test.drop(['tripid','pickup_time', 'drop_time', 'pickup_date', 'drop_date', 'pick_lat', 'pick_lon', 'drop_lat', 'drop_lon'], axis = 1)


### <span style="color:#4ec57a">Prepare train and test data sets

In [144]:
x_train = C_train.iloc[:,C_train.columns!='label_bool']
y_train = C_train['label_bool'].values
x_test = C_test

### <span style="color:#4ec57a">Random Forest Model and Prediction

In [145]:
rf = RandomForestRegressor()
rf.fit(x_train, y_train)
rf_predict = rf.predict(x_test)

### <span style="color:#4ec57a">Prepare for submission

In [146]:
sample_submission_random_forest = test.iloc[:, lambda df: [0]]
sample_submission_random_forest['label_bool'] = rf_predict

sample_submission_random_forest['prediction']=sample_submission_random_forest['label_bool'].apply(lambda x:1 if x >= 0.5 else 0)

sample_submission_random_forest = sample_submission_random_forest.drop(['label_bool'], axis = 1)
sample_submission_random_forest.to_csv('sample_submission_random_forest10.csv', index=False)

sample_submission_random_forest.head(5)

,tripid,prediction
0,213284604,1
1,213286352,1
2,213293973,1
3,213294622,1
4,213298687,1


# <span style="color:#c54e5e">Method 2: Classification Using LIGHTGBM</span>

##### <span style="color:#4ec57a">In this method I'm trying to predict the correctnes value which will be between 0 and 1. Prediction will be evaluated based on the value >= 0.5 or not</span>


In [20]:
lightgbm_params = {
        'boosting_type':'gbdt',
        'objective': 'regression',
        'nthread': -1,
        'verbose': 0,
        'num_leaves': 80,
        'learning_rate': 0.05,
        'max_depth': 7,
        'subsample': 0.8,
        'subsample_freq': 1,
        'colsample_bytree': 0.6,
        'reg_aplha': 1,
        'reg_lambda': 0.001,
        'metric': 'rmse',
        'min_split_gain': 0.5,
        'min_child_weight': 1,
        'min_child_samples': 10,
        'scale_pos_weight':1     
    }

In [21]:
pred_test_y = np.zeros(x_test.shape[0])

In [22]:
train_set = lgbm.Dataset(x_train, y_train, silent=True)

In [23]:
lgbm_model = lgbm.train(lightgbm_params, train_set = train_set, num_boost_round=3000)

In [24]:
pred_test_y = lgbm_model.predict(x_test, num_iteration = lgbm_model.best_iteration)

In [25]:
sample_submission_lgbm = test.iloc[:, lambda df: [0]]
sample_submission_lgbm['label_bool'] = pred_test_y
sample_submission_lgbm.head(20)

sample_submission_lgbm['prediction']=sample_submission_lgbm['label_bool'].apply(lambda x:1 if x >= 0.5 else 0)

sample_submission_lgbm = sample_submission_lgbm.drop(['label_bool'], axis = 1)
sample_submission_lgbm.to_csv('sample_submission_lgbm3.csv', index=False)

sample_submission_lgbm.head(5)

,tripid,prediction
0,213284604,1
1,213286352,1
2,213293973,1
3,213294622,1
4,213298687,1


# <span style="color:#c54e5e">Method 3: Classification Using Bayesian Optimization with XGBM</span>

##### <span style="color:#4ec57a">In this method I'm trying to predict the correctnes value which will be between 0 and 1. Prediction will be evaluated based on the value >= 0.5 or not</span>

In [56]:
import xgboost as xgb
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [57]:
#dtrain = xgb.DMatrix(x_train, label=y_train)
#dtest = xgb.DMatrix(x_test)

In [58]:
X_train, X_test, y_train, y_test = train_test_split(C_train.drop('label_bool', axis=1),
                                                    C_train['label_bool'], test_size=0.25)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test)


In [59]:
def xgb_evaluate(max_depth, gamma, colsample_bytree):
    params = {'eval_metric': 'rmse',
              'max_depth': int(max_depth),
              'subsample': 0.8,
              'eta': 0.1,
              'gamma': gamma,
              'colsample_bytree': colsample_bytree}
    # Used around 1000 boosting rounds in the full model
    cv_result = xgb.cv(params, dtrain, num_boost_round=100, nfold=3)    
    
    # Bayesian optimization only knows how to maximize, not minimize, so return the negative RMSE
    return -1.0 * cv_result['test-rmse-mean'].iloc[-1]

xgb_bo = BayesianOptimization(xgb_evaluate, {'max_depth': (3, 7), 
                                             'gamma': (0, 1),
                                             'colsample_bytree': (0.3, 0.9)})
# Use the expected improvement acquisition function to handle negative numbers
# Optimally needs quite a few more initiation points and number of iterations
xgb_bo.maximize(init_points=3, n_iter=5, acq='ei')

best_params = max(xgb_bo.res, key=lambda x:x['target'])['params']
print(best_params)
#params = xgb_bo.res['max']['max_params']
best_params['max_depth'] = int(best_params['max_depth'])

|   iter    |  target   | colsam... |   gamma   | max_depth |
-------------------------------------------------------------
|  1        | -0.2118   |  0.6913   |  0.7698   |  6.284    |
|  2        | -0.2142   |  0.6261   |  0.6437   |  5.274    |
|  3        | -0.219    |  0.8505   |  0.2476   |  3.437    |
|  4        | -0.2228   |  0.3176   |  0.9502   |  6.976    |
|  5        | -0.2083   |  0.9      |  0.0      |  6.997    |
|  6        | -0.2118   |  0.8935   |  0.9361   |  6.104    |
|  7        | -0.2088   |  0.8886   |  0.000746 |  6.007    |
|  8        | -0.2087   |  0.89     |  0.04319  |  6.245    |
{'colsample_bytree': 0.9, 'gamma': 0.0, 'max_depth': 6.996546589964646}


In [61]:
# Train a new model with the best parameters from the search
model2 = xgb.train(best_params, dtrain, num_boost_round=250)

# Predict on testing and training set
y_pred = model2.predict(dtest)
y_train_pred = model2.predict(dtrain)

In [62]:
dtest = xgb.DMatrix(C_test)

y_pred_xgb = model2.predict(dtest)

In [63]:
#xb = xgb.train(xgbm_params, dtrain, xgbm_num_rounds)


In [64]:
#y_pred_xgb = xb.predict(dtest)

In [65]:
sample_submission_xgbm = test.iloc[:, lambda df: [0]]
sample_submission_xgbm['label_bool'] = y_pred_xgb
sample_submission_xgbm.head(20)

sample_submission_xgbm['prediction']=sample_submission_xgbm['label_bool'].apply(lambda x:1 if x >= 0.5 else 0)

sample_submission_xgbm = sample_submission_xgbm.drop(['label_bool'], axis = 1)
sample_submission_xgbm.to_csv('sample_submission_xgbm8.csv', index=False)

sample_submission_xgbm.head(5)

,tripid,prediction
0,213284604,1
1,213286352,1
2,213293973,1
3,213294622,1
4,213298687,1


# <span style="color:#c54e5e">Method 4: Classification Deep learning</span>

##### <span style="color:#4ec57a">In this method I'm trying to predict the correctnes value which will be between 0 and 1. Prediction will be evaluated based on the value >= 0.5 or not</span>

In [18]:
train_df, validation_df = train_test_split(C_train, test_size=0.10, random_state=1)

# Get labels
train_labels = train_df['label_bool'].values
validation_labels = validation_df['label_bool'].values
train_df = train_df.drop(['label_bool'], axis=1)
validation_df = validation_df.drop(['label_bool'], axis=1)

In [20]:
test_clean = C_test

# Scale data

scaler = preprocessing.MinMaxScaler()
train_df_scaled = scaler.fit_transform(train_df)
validation_df_scaled = scaler.transform(validation_df)
test_scaled = scaler.transform(test_clean)

In [31]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical
from keras import regularizers


from keras.layers.normalization import BatchNormalization
from keras.optimizers import adam

In [46]:
# Model parameters
BATCH_SIZE = 128
LEARNING_RATE = 0.01
EPOCHS = 40

model = Sequential()
model.add(Dense(256, activation='relu', input_dim=train_df_scaled.shape[1], activity_regularizer=regularizers.l1(0.01)))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(8, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1))

adam = keras.optimizers.Adam(lr=LEARNING_RATE)  #keras.optimizers.adam(lr=LEARNING_RATE)
model.compile(loss='mse', optimizer="adam", metrics=['mae'])

In [47]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_68 (Dense)             (None, 256)               5632      
_________________________________________________________________
batch_normalization_56 (Batc (None, 256)               1024      
_________________________________________________________________
dense_69 (Dense)             (None, 128)               32896     
_________________________________________________________________
batch_normalization_57 (Batc (None, 128)               512       
_________________________________________________________________
dense_70 (Dense)             (None, 64)                8256      
_________________________________________________________________
batch_normalization_58 (Batc (None, 64)                256       
_________________________________________________________________
dense_71 (Dense)             (None, 32)              

In [48]:
history = model.fit(x=train_df_scaled, y=train_labels, batch_size=BATCH_SIZE, epochs=EPOCHS, 
                    verbose=1, validation_data=(validation_df_scaled, validation_labels), 
                    shuffle=True)

Train on 15270 samples, validate on 1697 samples
Epoch 1/40
15270/15270 [==============================] - 3s 184us/step - loss: 3.6802 - mae: 0.7176 - val_loss: 0.2881 - val_mae: 0.4945
Epoch 2/40
15270/15270 [==============================] - 1s 49us/step - loss: 0.1881 - mae: 0.3583 - val_loss: 0.1139 - val_mae: 0.2426
Epoch 3/40
15270/15270 [==============================] - 1s 51us/step - loss: 0.1066 - mae: 0.2010 - val_loss: 0.0955 - val_mae: 0.1790
Epoch 4/40
15270/15270 [==============================] - 1s 54us/step - loss: 0.0974 - mae: 0.1715 - val_loss: 0.0929 - val_mae: 0.1961
Epoch 5/40
15270/15270 [==============================] - 1s 54us/step - loss: 0.0945 - mae: 0.1672 - val_loss: 0.0957 - val_mae: 0.2238
Epoch 6/40
15270/15270 [==============================] - 1s 53us/step - loss: 0.0925 - mae: 0.1667 - val_loss: 0.0930 - val_mae: 0.1061
Epoch 7/40
15270/15270 [==============================] - 1s 54us/step - loss: 0.0942 - mae: 0.1684 - val_loss: 0.1023 - val_mae

In [49]:
prediction = model.predict(test_scaled, batch_size=128, verbose=1)


8576/8576 [==============================] - 1s 78us/step


In [51]:
type(prediction) 

numpy.ndarray

In [53]:
sample_submission_nn = test.iloc[:, lambda df: [0]]
sample_submission_nn['label_bool'] = prediction

sample_submission_nn['prediction']=sample_submission_nn['label_bool'].apply(lambda x:1 if x >= 0.5 else 0)

sample_submission_nn = sample_submission_nn.drop(['label_bool'], axis = 1)
sample_submission_nn.to_csv('sample_submission_nn.csv', index=False)

sample_submission_nn.head(5)

,tripid,prediction
0,213284604,1
1,213286352,1
2,213293973,1
3,213294622,1
4,213298687,1
